In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def rnd(shape):
    return np.random.random(shape)
%matplotlib inline

In [ ]:
x = np.linspace(0, 10, 50)
err = rnd(x.shape)
err_m = 10
heteroscedastic=0
homoscedastic=1
y = ((np.pi + (err_m*(rnd(x.shape)-0.5)*err)*heteroscedastic)*x + 
     np.exp(1) + (err_m*(rnd(x.shape)-0.5)*err)*homoscedastic)

In [ ]:
plt.plot(x, y, '.')

In [ ]:
def OLS(x, y):
    X = np.ones((x.size, 2))
    X[:, 1] = x
    beta = np.linalg.lstsq(X, y)[0]
    eps = X.dot(beta) - y
    SSR = eps.T.dot(eps) # sum of squared residuals
    vcv = SSR/(x.size - 2)*np.linalg.inv(X.T.dot(X))
    TSS = np.sum(np.square(y - np.mean(y)))
    R2 = 1 - SSR/TSS
    return beta, vcv, R2
    XtX = X.T.dot(X)
    return np.linalg.lstsq(XtX, X.T.dot(y))[0]

In [ ]:
def model(x, beta):
    X = np.ones((x.size, 2))
    X[:, 1] = x
    return X.dot(beta)

In [ ]:
beta, vcv, R2 = OLS(x, y)
plt.errorbar(x, y-model(x, beta), yerr=err, marker='.', ls='None')
print(R2)